In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import os
import json

## 1. Load data

In [2]:
ALL_FEATURE_DIR = './data/all_features/'

In [3]:
REGION = 'euw1'
TIER = 'CHALLENGER'
DIVISION = 'I'

In [4]:
df = pd.read_parquet(os.path.join(ALL_FEATURE_DIR, f'all_features_{REGION}_{TIER}_{DIVISION}.parquet'))

In [5]:
df = df[['metadata_match_id', 'puuid', 'units', 'active_traits', 'placement']]

In [6]:
df.head()

,metadata_match_id,puuid,units,active_traits,placement
0,EUW1_7287829871,25LytTrGnuEzDLLQKSFQjKtDtzdWiYqMYEPImOuwbISTLG...,"[{'unit_name': 'Zyra', 'unit_cost': 1, 'unit_t...","[{'trait_name': 'Emissary', 'trait_style': 1.0...",4
1,EUW1_7287021125,RYLtU0PFAnpFjJ0pJf_i1P_hiMh9w2SF9xxl4Dz4SaiGk9...,"[{'unit_name': 'Irelia', 'unit_cost': 1, 'unit...","[{'trait_name': 'Academy', 'trait_style': 1.0,...",6
2,EUW1_7285135299,q-EzF1r80j1TaaYVCjh1M_gQIL1_GjHt5s5mDT5J0EMFMV...,"[{'unit_name': 'Zyra', 'unit_cost': 1, 'unit_t...","[{'trait_name': 'Bruiser', 'trait_style': 1.0,...",1
3,EUW1_7287517140,dMaMJ-O8aJW2DmjNVIq588fDocoq-p7TnPP6tT0y3Cy3bS...,"[{'unit_name': 'Powder', 'unit_cost': 1, 'unit...","[{'trait_name': 'Ambusher', 'trait_style': 4.0...",1
4,EUW1_7279610050,vmj1lvV5X0BLkh0RMzBBYH-L2QYDekEC40D5-6Z2iPW9tK...,"[{'unit_name': 'Morgana', 'unit_cost': 1, 'uni...","[{'trait_name': 'Visionary', 'trait_style': 2....",8


In [7]:
df.iloc[1]['units']

array([{'unit_name': 'Irelia', 'unit_cost': 1, 'unit_tier': 1, 'unit_item_count': 0, 'players_contesting_unit': 0.0, 'players_contesting_unit_tier_equal': 0.0, 'players_contesting_unit_tier_higher': 0.0},
       {'unit_name': 'Leona', 'unit_cost': 2, 'unit_tier': 2, 'unit_item_count': 1, 'players_contesting_unit': 1.0, 'players_contesting_unit_tier_equal': 0.0, 'players_contesting_unit_tier_higher': 1.0},
       {'unit_name': 'Rell', 'unit_cost': 2, 'unit_tier': 2, 'unit_item_count': 0, 'players_contesting_unit': 1.0, 'players_contesting_unit_tier_equal': 0.0, 'players_contesting_unit_tier_higher': 1.0},
       {'unit_name': 'Ezreal', 'unit_cost': 3, 'unit_tier': 3, 'unit_item_count': 3, 'players_contesting_unit': 0.0, 'players_contesting_unit_tier_equal': 0.0, 'players_contesting_unit_tier_higher': 0.0},
       {'unit_name': 'Loris', 'unit_cost': 3, 'unit_tier': 2, 'unit_item_count': 0, 'players_contesting_unit': 3.0, 'players_contesting_unit_tier_equal': 2.0, 'players_contesting_unit

In [20]:
test_data = df.iloc[1]['units']

def calculate_unit_strength(unit):
    alpha = 1
    beta = 1
    epsilon = 1e-8
    unit_strength = alpha * ((unit['unit_cost']+1)**(unit['unit_tier']-1) * (1+np.log(unit['unit_item_count']+1)))
    unit_contest = beta * (np.log(1 + unit['players_contesting_unit']) + np.log(unit['players_contesting_unit_tier_equal']+1) + np.log(unit['players_contesting_unit_tier_equal']+1)**2)
    return unit_strength - unit_contest

unit_strength = [calculate_unit_strength(x) for x in test_data]
print(unit_strength)
print(sum(unit_strength))

[1.0, 4.386294361119891, 2.3068528194400546, 38.18070977791825, 0.3081443893994176, 8.62631406830246, 8.239616194998872, 2.09861228866811, 1.0]
66.14654389984706


In [18]:
np.log(7)

1.9459101490553132

## 2. Embed units using weighted pooling

In [8]:
# get all unique unit names
with open('./data/data_dragon/set13/units.json', 'r') as file:
    units = json.load(file)

In [9]:
SET13_UNITS = [i['name'] for i in units.values()]

len(SET13_UNITS)

63

In [10]:
SPECIAL_UNITS = ['Black Rose Sion', "Jayce's summons"]

In [59]:
# class TFTUnitEmbedder:
#     def __init__(self, 
#                  # unique_units=SET13_UNITS, 
#                  # special_units=SPECIAL_UNITS, 
#                  embedding_dim=8, 
#                  alpha_strength=0.5, 
#                  alpha_contest=0.3, 
#                  epsilon=1e-8):
#         # self.unit_to_idx = {unit: i for i, unit in enumerate(sorted(unique_units))}
#         # for unit in special_units:
#         #     self.unit_to_idx[unit] = len(self.unit_to_idx)
#         self.embedding_dim = embedding_dim
#         self.alpha_strength = alpha_strength    # scaling factor for unit cost & tier & items
#         self.alpha_contest = alpha_contest      # scaling factor for contesting pressure
#         self.epsilon = epsilon
#         # create embedding layer
#         # self.embedding_layer = nn.Embedding(len(unique_units)+len(special_units), embedding_dim)

#     def get_pooled_embedding(self, units):
#         if len(units) == 0:
#             return torch.zeros(self.embedding_dim)
#         # convert unit names to indices
#         # unit_idx = torch.tensor([self.unit_to_idx.get(unit['unit_name'], -1) for unit in units], dtype=torch.long)
#         # unit_embeddings = self.embedding_layer(unit_idx)
#         # extract other numerical features
#         unit_costs = torch.tensor([unit['unit_cost'] for unit in units], dtype=torch.float)
#         unit_tiers = torch.tensor([unit['unit_tier'] for unit in units], dtype=torch.float)
#         unit_items = torch.tensor([unit['unit_item_count'] for unit in units], dtype=torch.float)
#         players_contesting = torch.tensor([unit['players_contesting_unit'] for unit in units], dtype=torch.float)
#         players_contesting_tier_higher = torch.tensor([unit['players_contesting_unit_tier_higher'] for unit in units], dtype=torch.float)

#         # compute weights with separate scaling factors
#         strength_component = self.alpha_strength * (unit_costs * 3**(unit_tiers-1) + (unit_items+self.epsilon) / 3) 
#         contest_component = self.alpha_contest * ((players_contesting_tier_higher+self.epsilon) / (players_contesting+self.epsilon))
#         weights = strength_component + contest_component
#         # normalize weights
#         weights = weights / weights.sum() if weights.sum() > 0 else weights
#         # apply weighted pooling
#         pooled_embedding = torch.max(weights, dim=0)
#         return pooled_embedding

#     def transform(self, df, unit_col='units'):
#         # compute pooled embeddings for all rows
#         df['units_embedding'] = df[unit_col].apply(lambda units: self.get_pooled_embedding(units))
#         return df

In [60]:
unit_emb = TFTUnitEmbedder(embedding_dim=4, alpha_strength=0.5, alpha_contest=1.0)

In [61]:
df = unit_emb.transform(df)

In [62]:
df.head()

,metadata_match_id,puuid,units,active_traits,placement,units_embedding
0,EUW1_7287829871,25LytTrGnuEzDLLQKSFQjKtDtzdWiYqMYEPImOuwbISTLG...,"[{'unit_name': 'Zyra', 'unit_cost': 1, 'unit_t...","[{'trait_name': 'Emissary', 'trait_style': 1.0...",4,"(tensor(0.2093), tensor(1))"
1,EUW1_7287021125,RYLtU0PFAnpFjJ0pJf_i1P_hiMh9w2SF9xxl4Dz4SaiGk9...,"[{'unit_name': 'Irelia', 'unit_cost': 1, 'unit...","[{'trait_name': 'Academy', 'trait_style': 1.0,...",6,"(tensor(0.3203), tensor(3))"
2,EUW1_7285135299,q-EzF1r80j1TaaYVCjh1M_gQIL1_GjHt5s5mDT5J0EMFMV...,"[{'unit_name': 'Zyra', 'unit_cost': 1, 'unit_t...","[{'trait_name': 'Bruiser', 'trait_style': 1.0,...",1,"(tensor(0.1950), tensor(1))"
3,EUW1_7287517140,dMaMJ-O8aJW2DmjNVIq588fDocoq-p7TnPP6tT0y3Cy3bS...,"[{'unit_name': 'Powder', 'unit_cost': 1, 'unit...","[{'trait_name': 'Ambusher', 'trait_style': 4.0...",1,"(tensor(0.1957), tensor(6))"
4,EUW1_7279610050,vmj1lvV5X0BLkh0RMzBBYH-L2QYDekEC40D5-6Z2iPW9tK...,"[{'unit_name': 'Morgana', 'unit_cost': 1, 'uni...","[{'trait_name': 'Visionary', 'trait_style': 2....",8,"(tensor(0.3182), tensor(5))"


In [57]:
df[df['metadata_match_id'] == 'EUW1_7287829871'].head(8)

,metadata_match_id,puuid,units,active_traits,placement,units_embedding
0,EUW1_7287829871,25LytTrGnuEzDLLQKSFQjKtDtzdWiYqMYEPImOuwbISTLG...,"[{'unit_name': 'Zyra', 'unit_cost': 1, 'unit_t...","[{'trait_name': 'Emissary', 'trait_style': 1.0...",4,"([tensor(0.2093)], [tensor(1)])"
6713,EUW1_7287829871,luS03nUnioqAHW4JTjCnMJozDNFxhQZ15QnxMt1A1RuseY...,"[{'unit_name': 'Tristana', 'unit_cost': 2, 'un...","[{'trait_name': 'Emissary', 'trait_style': 4.0...",2,"([tensor(0.1779)], [tensor(8)])"
7567,EUW1_7287829871,dW0e-sSivzXayv2b9M1RKBl2U4R_p9-8zsWm0mb_QIATQH...,"[{'unit_name': 'Vladimir', 'unit_cost': 2, 'un...","[{'trait_name': 'Emissary', 'trait_style': 4.0...",6,"([tensor(0.2155)], [tensor(6)])"
18678,EUW1_7287829871,ajpljYRv-8pKhLjOMblvF2pgOVTOaHbuGxXYmpHCaywED5...,"[{'unit_name': 'Morgana', 'unit_cost': 1, 'uni...","[{'trait_name': 'Visionary', 'trait_style': 4....",5,"([tensor(0.2377)], [tensor(4)])"
22504,EUW1_7287829871,_JcFIAwTDEEQTWxTrxt30xaLaioPAyFXPucawYMIY9oDm5...,"[{'unit_name': 'Powder', 'unit_cost': 1, 'unit...","[{'trait_name': 'Emissary', 'trait_style': 1.0...",8,"([tensor(0.1775)], [tensor(6)])"
24207,EUW1_7287829871,2E8YEKoXEHXanZWKaxVT4K6KT8rfeZyDK4WE19UnVGrRYa...,"[{'unit_name': 'Vladimir', 'unit_cost': 2, 'un...","[{'trait_name': 'Emissary', 'trait_style': 4.0...",3,"([tensor(0.1798)], [tensor(7)])"
25792,EUW1_7287829871,qwIGHwnoGf-NeE2wE2jdeLT1v5ALnSPCokUIqvEeJ2-pUJ...,"[{'unit_name': 'Trundle', 'unit_cost': 1, 'uni...","[{'trait_name': 'Blood Hunter', 'trait_style':...",1,"([tensor(0.1479)], [tensor(7)])"
25793,EUW1_7287829871,Dtl3sdyemYrVqK6k-QnVPEvCWHeVkfUtDfP8Ea5ijlQa_U...,"[{'unit_name': 'Lux', 'unit_cost': 1, 'unit_ti...","[{'trait_name': 'Emissary', 'trait_style': 1.0...",7,"([tensor(0.2038)], [tensor(5)])"


In [58]:
df.loc[(df['metadata_match_id'] == 'EUW1_7287829871') & (df['placement']==1)].iloc[0]['units_embedding']

torch.return_types.max(
values=tensor([0.1479]),
indices=tensor([7]))

In [51]:
df.loc[(df['metadata_match_id'] == 'EUW1_7287829871') & (df['placement']==2)].iloc[0]['units_embedding']

tensor([1.])

In [52]:
df.loc[(df['metadata_match_id'] == 'EUW1_7287829871') & (df['placement']==8)].iloc[0]['units_embedding']

tensor([1.])